In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sentiment Analysis for large scale data using LLM (PaLM API in Vertex AI)

## Overview

This notebook shows how to perform sentimental analysis on large scale data using LLM.
The dataset used is a public dataset from Bigquery Public Datasets.

#### **Steps**
Using Spark, 
1) This notebook reads data from Bigquery public dataset **bigquery-public-data.imdb.reviews**
2) It calls [Vertex AI Text Bison](https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/api-quickstart#try_text_prompts) to find the sentiment of each review (positive vs negative)
3) We compare the result side by side
4) Find accuracy, and again trim the input and observe the accuracy increase

#### Related content

- [Text Prompt](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-prompts)
- [Content Classification](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-prompts#content-classification)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer

import google.auth
import google.auth.transport.requests
import requests

import time

#### Get credentials to authenticate with Google APIs


In [ ]:
credentials, project_id = google.auth.default()
auth_req = google.auth.transport.requests.Request()
credentials.refresh(auth_req)

### Create Spark Session for the notebook

In [ ]:
spark = SparkSession.builder \
    .appName("Sentimental Analysis using Dataproc and Vertex LLM") \
    .getOrCreate()

### Read data from Bigquery Public Dataset 

In [ ]:
movie_reviews = spark.read.format("bigquery").option("table", "bigquery-public-data.imdb.reviews").load()

|                                                                                              review|split|   label| movie_id|reviewer_rating|                           movie_url|title|
|----------------------------------------------------------------------------------------------------|-----|--------|---------|---------------|------------------------------------|-----|
|I had to see this on the British Airways plane. It was terribly bad acting and a dumb story. Not ...| test|Negative|tt0158887|              2|http://www.imdb.com/title/tt0158887/| null|
|This is a family movie that was broadcast on my local ITV station at 1.00 am a couple of nights a...| test|Negative|tt0158887|              4|http://www.imdb.com/title/tt0158887/| null|
|I would like to comment on how the girls are chosen. why is that their are always more white wome...| test|Negative|tt0391576|              2|http://www.imdb.com/title/tt0391576/| null|
|Tyra & the rest of the modeling world needs to know that real women like myself and my daughter d...| test|Negative|tt0391576|              3|http://www.imdb.com/title/tt0391576/| null|

### Get Positive Reviews from Dataset

In [ ]:
positive_movie_reviews = movie_reviews.select(col("review"), col("reviewer_rating"), col("movie_id"), col("label")).where(col("label") == "Positive").limit(100)

### Get Negative Reviews from Dataset

In [ ]:
negative_movie_reviews = movie_reviews.select(col("review"), col("reviewer_rating"), col("movie_id"), col("label")).where(col("label") == "Negative").limit(100)

### Mix positive and negative 
Making union of positive and negative reviews to get a good dataset of mixed set of reviews. For the purpose notebook, each class of reviews has 100 rows each.

In [ ]:
movie_reviews_mixed = positive_movie_reviews.union(negative_movie_reviews)

|              review|reviewer_rating| movie_id|   label|
|--------------------|---------------|---------|--------|
|This movie is ama...|             10|tt0187123|Positive|
|THE HAND OF DEATH...|             10|tt0187123|Positive|
|The Hand of Death...|              7|tt0187123|Positive|
|Just as a reminde...|             10|tt0163955|Positive|
|Like an earlier c...|              9|tt0163955|Positive|

### Final count is 200 as can be seen below

In [ ]:
movie_reviews_mixed.count()

### Creating a UDF to get predictions from Vertext Text Bison Model
In this method, text whose sentiment is to be predicted is passed. Model used is text-bison-32k which allow 32k input tokens, to accomodate bigger text inputs of movie reviews. 

In [ ]:
def find_sentiment_zero_shot(text):
    
    def vertex_get_prediction(prompt):
        MODEL_ID="text-bison-32k"
        prediction = requests.post(
            f"https://us-central1-aiplatform.googleapis.com/v1/projects/{project_id}/locations/us-central1/publishers/google/models/{MODEL_ID}:predict",
            headers={'Authorization': 'Bearer %s' % credentials.token,
                     'Content-Type': 'application/json'},
            json = {
                      "instances": [
                        { "prompt": prompt}
                      ],
                      "parameters": {
                        "temperature": 0.2,
                        "maxOutputTokens": 256,
                        "topK": 40,
                        "topP": 0.95
                    }
            }
        ).json()
        print(prediction)

        if "predictions" in prediction:
            pred = prediction["predictions"][0]
            if "content" in pred:
                return pred["content"]
        else:
            if "error" in prediction:
                if prediction["error"]["code"] == 429:  # Quota exceeded
                    time.sleep(5)
                    return vertex_get_prediction(prompt)
                else:
                    return f"Error getting prediction: {prediction['error']}"

            return f"Error getting predictions"
        
    prompt = f"""For the given text below, provide the sentiment classification from the two classes mentioned below:
    The two classes are: Negative, Positive.
    Always choose between one of them (the most appropriate one.
    Text: {text}
    Sentiment:"""
    
    sentiment = vertex_get_prediction(prompt)
        
    return sentiment

In [ ]:
find_sentiment_zero_shot_udf = udf(find_sentiment_zero_shot)

In [ ]:
movie_reviews_mixed.printSchema()

### Get prediction from the LLM using the UDF on the movie reviews

In [ ]:
movie_review_sentiment_pred = movie_reviews_mixed.withColumn("predicted_sentiment", find_sentiment_zero_shot_udf(movie_reviews_mixed["review"])).cache()

In [ ]:
# Trim whitespaces
trimmed_movie_review_sentiment_pred = movie_review_sentiment_pred.withColumn("predicted_sentiment", trim(col("predicted_sentiment"))).withColumn("label", trim(col("label")))

### Let's check the predicted value and do a quick comparison of required output v/s actual label

In [ ]:
trimmed_movie_review_sentiment_pred.select(col("predicted_sentiment"), col("label")).show(truncate=False)

### Evaluation

Let's index the classes Negative and Positive to 1 and 0.  

In [ ]:
inputs = ["predicted_sentiment", "label"]
outputs = ["predicted_sentiment_indexed", "label_indexed"]

stringIndexer = StringIndexer(inputCols=inputs, outputCols=outputs)
indexer = stringIndexer.fit(trimmed_movie_review_sentiment_pred)

movie_review_sentiment_pred_indexed = indexer.transform(trimmed_movie_review_sentiment_pred)

And use the BinaryClassificationEvaluator to output our Area Under the ROC curve (AUC-ROC)

In [ ]:
evaluator = BinaryClassificationEvaluator()
evaluator.setRawPredictionCol("predicted_sentiment_indexed")
evaluator.setLabelCol("label_indexed")

area_under_roc = evaluator.evaluate(movie_review_sentiment_pred_indexed, {evaluator.metricName: "areaUnderROC"})

print("area_under_roc (%): ", area_under_roc)

Without any prior training or chaining prompts, only by zero shot, the model has been able to predict sentiments properly with 86% AUC-ROC

#### Count the number of unsuccessful predictions

In [ ]:
match_predictions_df = movie_review_sentiment_pred_indexed.withColumn("if_match", when((col("predicted_sentiment_indexed")==col("label_indexed")),1).otherwise(0))

In [ ]:
match_predictions_df.where(col("if_match")==0).count()

#### Percentage Accuracy  

Total Rows = 200
* Mislabeled rows = 20
* Accuracy = (True positives + True Negatives)/ (True positives + True negatives + False positives + False negatives)
* Percentage Accuracy= 180 / 200 = 90%

#### Check the mismatch predictions
Find the mismatched rows and show it

In [ ]:
mismatch_df = match_predictions_df.where(col("if_match")==0).select(col('predicted_sentiment'),col('label'),col('review'))

In [ ]:
mismatch_df.show()